## GPTQModel Pipeline

### Install GPTQModel

In [1]:
# clone GPTQModel repo
!git clone --depth 1 --branch v0.9.9 https://github.com/ModelCloud/GPTQModel.git

# compile and install GPTQModel
!cd GPTQModel && pip install --no-build-isolation .

Cloning into 'GPTQModel'...
remote: Enumerating objects: 210, done.
remote: Counting objects: 100% (210/210), done.
remote: Compressing objects: 100% (176/176), done.
Receiving objects: 100% (210/210), 200.83 KiB | 10.57 MiB/s, done.
remote: Total 210 (delta 35), reused 90 (delta 28), pack-reused 0
Resolving deltas: 100% (35/35), done.
Note: switching to '519fbe3ef02335c58e3aa8e9353f8346a8780b91'.

You are in 'detached HEAD' state. You can look around, make experimental
changes and commit them, and you can discard any commits you make in this
state without impacting any branches by switching back to a branch.

If you want to create a new branch to retain commits you create, you may
do so (now or later) by using -c with the switch command. Example:

  git switch -c <new-branch-name>

Or undo this operation with:

  git switch -

Turn off this advice by setting config variable advice.detachedHead to false

Processing /content/GPTQModel
  Preparing metadata (setup.py) ... done
  Using cac

### Simple GPTQ Quantization

Using the WikiText2 dataset and microsoft/Phi-3-mini-128k-instruct.

In [ ]:
import torch
import logging
from gptqmodel import GPTQModel, QuantizeConfig
from transformers import AutoTokenizer
from datasets import load_dataset


pretrained_model_id = "teknium/OpenHermes-2-Mistral-7B"
quantized_model_id = "teknium/OpenHermes-2-Mistral-7B-4bit-128g"


def get_open_instruct(tokenizer, nsamples, seqlen):
    traindata = load_dataset("VMware/open-instruct", "default", split="train").filter(
        lambda x: len(x["response"]) >= seqlen
    )

    return [tokenizer(example["response"]) for example in traindata.select(range(nsamples))]


@torch.no_grad()
def calculate_avg_ppl(model, tokenizer):
    from gptqmodel.utils import Perplexity

    ppl = Perplexity(
        model=model,
        tokenizer=tokenizer,
        dataset_path="VMware/open-instruct",
        dataset_name="default",
        split="train",
        text_column="response",
    )

    # n_ctx is context size
    # n_batch is the batch size
    all = ppl.calculate(n_ctx=128, n_batch=128)

    # average ppl
    avg = sum(all) / len(all)

    return avg


def main():
    tokenizer = AutoTokenizer.from_pretrained(pretrained_model_id, use_fast=True)

    print("Loading Open Instruct training data...")
    train_dataset = get_open_instruct(tokenizer, nsamples=512, seqlen=1024)
    print("Completed loading of Open Instruct training data!")

    quantize_config = QuantizeConfig(
        # quantize model to 4-bit
        bits=4,
        # 128 offer good balance between inference speed and quantization quality
        # 32 will increase vRAM usage but increase inferencing quality
        group_size=32,
        # increase damp if NaN is encountered during `.quantize()` and/or increase calibration dataset size
        damp_percent=0.005,
        desc_act=True,
        static_groups=False,
        sym=True,
        true_sequential=True,
        lm_head=False,
        # marlin is vLLM's preferred GPTQ quantization method, which is included in "gptq"
        quant_method="gptq",
    )

    # load un-quantized model, the model will always be force loaded into cpu
    model = GPTQModel.from_pretrained(pretrained_model_id, quantize_config)

    print("Beginning quantization...")
    # quantize model, the calibration_dataset should be list of dict whose keys can only be "input_ids" and "attention_mask"
    # with value under torch.LongTensor type.
    model.quantize(train_dataset)
    print("Quantization complete!")

    print("Saving quantized model...")
    # save quantized model
    model.save_quantized(quantized_model_id)
    # save quantized model using safetensors
    model.save_quantized(quantized_model_id, use_safetensors=True)
    print("Saving quantized model complete!")

    # load quantized model, currently only support cpu or single gpu
    model = GPTQModel.from_quantized(quantized_model_id, device="cuda:0")

    # inference with model.generate
    print(
        tokenizer.decode(
            model.generate(
                **tokenizer("What is the capital of Jamaica?", return_tensors="pt").to(
                    "cuda:0"
                )
            )[0]
        )
    )

    print(
        f"Quantized Model {quantized_model_id} avg PPL is {calculate_avg_ppl(model, tokenizer)}"
    )

# set logging configuration for GPTQModel
logging.basicConfig(
    format="%(asctime)s %(levelname)s [%(name)s] %(message)s",
    level=logging.INFO,
    datefmt="%Y-%m-%d %H:%M:%S",
)

# execute main method
main()

Loading Open Instruct training data...
Completed loading of Open Instruct training data!


pytorch_model-00001-of-00002.bin:  10%|9         | 954M/9.94G [00:00<?, ?B/s]